# 2022/1/20
# 给刘肖林计算ceof

In [7]:
import os
import sys
import xarray as xr
import numpy as np
import scipy.linalg
module_path = ["/home/sun/mycode/module/","/data5/2019swh/mycode/module/"]
sys.path.append(module_path[0])
from module_sun import *
import cartopy.crs as ccrs

In [8]:
path       =  "/home/sun/data/ceof-lxl/"
f0         =  xr.open_dataset(path+"NH_oned_hibert_sf_1989_1990.nc")
file_list  =  os.listdir(path) ; file_list.sort()

matrix1  =  np.zeros((51,11280*40),dtype=complex)
for ff in range(len(file_list)):
    f1      =  xr.open_dataset(path+file_list[ff])
    number  =  f1.ZR.data + 1J*f1.ZI.data
    matrix1[:,ff*11280:(ff+1)*11280] = number

In [9]:
S   =  1/51*np.dot(matrix1,matrix1.T.conjugate())
values,vectors = scipy.linalg.eigh(S,np.identity(51))
 
space_matrix  =  np.dot(vectors.T.conjugate(),matrix1)

In [10]:
v1  =  np.dot(matrix1.T.conjugate(),vectors)/(51 ** 0.5)

matrix_eigen  =  np.zeros((51,51),dtype=complex)
for i in range(51):
    matrix_eigen[i,i]  =  values[i]

t1  =  matrix_eigen.copy()
t2  =  np.dot(matrix1,v1)
for i in range(51):
    t1[i,i]  =  t2[i,i] / (51**0.5)*matrix_eigen[i,i]

In [11]:
ncfile  =  xr.Dataset(
    {
        "v_real": (["axis1","axis2"], v1.real),
        "v_imag": (["axis1","axis2"], v1.imag),
        "t_real": (["axis2","axis2"], t1.real),
        "t_imag": (["axis2","axis2"], t1.imag),
    },
    coords={
        "axis1": (["axis1"], np.linspace(1,v1.shape[0],v1.shape[0])),
        "axis2": (["axis2"], np.linspace(1,v1.shape[1],v1.shape[1])),
    },
)
ncfile.attrs["desctiption"]  =  "for lxl ceof 2022-1-29"
ncfile.to_netcdf("/home/sun/data/ceof_out_lxl.nc")
